# Lab 7  Additional Spark SQL and Dataframe techniques (Total 20 marks)


In [0]:
# Managing Imports
from pyspark.sql.functions import collect_set, avg, round, to_date, col

In [0]:
# Loading all the initial DF's
mpgdf = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/mpg.csv")
flight_summary = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/flight_summary___Copy.csv")
airportsdf = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/airports.csv")
aapl_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/aapl_2017.csv")

1. (5 marks) The file mpg.csv contains data about various automobiles
Load this file into a dataframe.  Group by the manufacturer and use a collection method to add a column which shows all unique models that manufacturer has.

In [0]:

mpgdf.groupBy("manufacturer").agg(collect_set("model").alias("unique_models")).show(truncate=False)

+------------+--------------------------------------------------------------------------------------+
|manufacturer|unique_models                                                                         |
+------------+--------------------------------------------------------------------------------------+
|land rover  |[range rover]                                                                         |
|pontiac     |[grand prix]                                                                          |
|toyota      |[4runner 4wd, camry solara, corolla, land cruiser wagon 4wd, toyota tacoma 4wd, camry]|
|lincoln     |[navigator 2wd]                                                                       |
|audi        |[a4, a6 quattro, a4 quattro]                                                          |
|jeep        |[grand cherokee 4wd]                                                                  |
|dodge       |[durango 4wd, ram 1500 pickup 4wd, dakota pickup 4wd, caravan 2wd]  

2. (5 marks) For the dataframe loaded from the mpg.csv file, generate a dataframe which is grouped by the year and cylinder (cyl) columns.  Use the pivot method to add columns for the average city mpg(cty) for each manufacturer.  Round the average city mpg to 1 decimal place to make the output easier to read.


In [0]:
mpgdf.groupBy("year", "cyl").pivot("manufacturer").agg(round(avg("cty"), 1).alias("avg_city_mpg")).show(truncate=False)

+----+---+----+---------+-----+----+-----+-------+----+----------+-------+-------+------+-------+------+------+----------+
|year|cyl|audi|chevrolet|dodge|ford|honda|hyundai|jeep|land rover|lincoln|mercury|nissan|pontiac|subaru|toyota|volkswagen|
+----+---+----+---------+-----+----+-----+-------+----+----------+-------+-------+------+-------+------+------+----------+
|2008|8  |16.0|13.6     |11.9 |13.6|null |null   |11.8|12.0      |12.0   |13.0   |12.0  |16.0   |null  |13.5  |null      |
|1999|4  |18.3|19.0     |18.0 |null|24.8 |18.5   |null|null      |null   |null   |20.0  |null   |19.0  |20.0  |23.3      |
|1999|6  |16.2|18.0     |14.9 |15.3|null |18.0   |15.0|null      |null   |14.0   |16.5  |17.0   |null  |16.5  |16.8      |
|2008|6  |16.8|17.5     |15.1 |15.3|null |17.3   |16.0|null      |null   |13.0   |17.8  |18.0   |null  |16.8  |17.0      |
|2008|5  |null|null     |null |null|null |null   |null|null      |null   |null   |null  |null   |null  |null  |20.5      |
|2008|4  |20.0|2

3. (5 marks) The file flight-summary.csv contains information about flights between various locations.   The file airports.csv contains additional information about each airport, some of which is not in the flight-summary.csv file.

Create a dataframe which contains the origin_code, origin_airport, latitude (for origin airport), longitude (for origin airport), dest_code, dest_airport, latitude (for destination airport), longitude (for destination airport) for every flight which originates in the state of Texas (TX).  

Which type of join technique ( Shuffle or Broadcast) did Spark likely use for this exercise?  Why?

In [0]:
flight_summary.show()
airportsdf.show()

+-----------+--------------------+---------------+------------+---------+--------------------+----------------+----------+-----+
|origin_code|      origin_airport|    origin_city|origin_state|dest_code|        dest_airport|       dest_city|dest_state|count|
+-----------+--------------------+---------------+------------+---------+--------------------+----------------+----------+-----+
|        BQN|Rafael Hernández ...|      Aguadilla|          PR|      MCO|Orlando Internati...|         Orlando|        FL|  441|
|        PHL|Philadelphia Inte...|   Philadelphia|          PA|      MCO|Orlando Internati...|         Orlando|        FL| 4869|
|        MCI|Kansas City Inter...|    Kansas City|          MO|      IAH|George Bush Inter...|         Houston|        TX| 1698|
|        SPI|Abraham Lincoln C...|    Springfield|          IL|      ORD|Chicago O'Hare In...|         Chicago|        IL|  998|
|        SNA|John Wayne Airpor...|      Santa Ana|          CA|      PHX|Phoenix Sky Harbo...|   

In [0]:
from pyspark.sql.functions import broadcast

joined_df = flight_summary \
    .join(broadcast(airportsdf).alias("origin"), col("origin_code") == col("origin.IATA_CODE"), "left") \
    .join(broadcast(airportsdf).alias("dest"), col("dest_code") == col("dest.IATA_CODE"), "left") \
    .filter(col("origin.state") == "TX") \
    .select(
        col("origin_code"),
        col("origin.airport").alias("origin_airport"),
        col("origin.latitude").alias("origin_latitude"),
        col("origin.longitude").alias("origin_longitude"),
        col("dest_code"),
        col("dest.airport").alias("dest_airport"),
        col("dest.latitude").alias("dest_latitude"),
        col("dest.longitude").alias("dest_longitude")
    )



joined_df.show()

+-----------+--------------------+---------------+----------------+---------+--------------------+-------------+--------------+
|origin_code|      origin_airport|origin_latitude|origin_longitude|dest_code|        dest_airport|dest_latitude|dest_longitude|
+-----------+--------------------+---------------+----------------+---------+--------------------+-------------+--------------+
|        LBB|Lubbock Preston S...|       33.66364|      -101.82278|      DEN|Denver Internatio...|     39.85841|      -104.667|
|        AUS|Austin-Bergstrom ...|       30.19453|       -97.66987|      ELP|El Paso Internati...|     31.80667|    -106.37781|
|        HOU|William P. Hobby ...|       29.64542|       -95.27889|      PDX|Portland Internat...|     45.58872|     -122.5975|
|        DFW|Dallas/Fort Worth...|       32.89595|        -97.0372|      PNS|Pensacola Interna...|     30.47331|     -87.18744|
|        DFW|Dallas/Fort Worth...|       32.89595|        -97.0372|      SDF|Louisville Intern...|     3

4. (5 marks) The file aapl-2017.csv contains information about daily apple stock prices.  Calculate the weekly average price using a window function inside the groupBy transformation.  Order the result by the start time.

In [0]:
from pyspark.sql.functions import col, weekofyear, avg, to_date

# Convert 'date' column to DateType
aapl_df = aapl_df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

# Group by the week number and calculate the average price
aapl_df.groupBy(weekofyear(col("date")).alias("week_number")).agg(avg(col("close")).alias("weekly_avg_price")).orderBy("week_number").show(truncate=False)


+-----------+------------------+
|week_number|weekly_avg_price  |
+-----------+------------------+
|1          |116.67250100000001|
|2          |119.228           |
|3          |119.94249925      |
|4          |121.16399980000001|
|5          |125.86799920000001|
|6          |131.6799956       |
|7          |134.97799980000002|
|8          |136.75000025      |
|9          |138.4899994       |
|10         |139.1359984       |
|11         |139.8660032       |
|12         |140.8559996       |
|13         |143.278           |
|14         |143.898001        |
|15         |141.91250225      |
|16         |141.68399959999996|
|17         |143.85799559999998|
|18         |147.32800020000002|
|19         |154.0619996       |
|20         |153.40399779999998|
+-----------+------------------+
only showing top 20 rows

